# Importing necessary packages and libraries

In [120]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen, Request
import pandas as pd
import datetime

# Getting data from alphavantage with the specified parameters

In [824]:
function="NEWS_SENTIMENT"
ticker = "ORCL"
topics="technology"
api_key="W9R35ZPC9W9DTSD0"
limit="1000"
time_from="20230710T1001"
sort="EARLIEST"

url = 'https://www.alphavantage.co/query?function='+function+'&tickers='+ticker+'&topics='+topics+'&apikey='+api_key+'&limit='+limit+'&time_from='+time_from+'&sort='+sort
r = requests.get(url)
data = r.json()
df = pd.DataFrame(data['feed'])

#converting the timestamp into a date time format
df['time_published'] = pd.to_datetime(df['time_published'])

# Extracting 'time_published' into seperate date and time columns

In [825]:
# Convert the 'time_published' column to string (if it's not already a string)
df['time_published'] = df['time_published'].astype(str)

# Split the 'time_published' column into 'Date' and 'Time' columns
df[['Date', 'Time']] = df['time_published'].str.split(' ', 1, expand=True)

# Convert the 'Date' and 'Time' columns into datetime objects
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time

# Drop the original 'time_published' column if needed
df.drop('time_published', axis=1, inplace=True)
# df.head()

# Removing unwanted columns

In [826]:
#removing unwanted columns
columns_to_remove = ['url', 'authors','banner_image','source', 'category_within_source', 'source_domain', 'topics']
df.drop(columns_to_remove, axis=1, inplace=True)

df.head()

,title,summary,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,Date,Time
0,"Beat the Market Like Zacks: Shopify, Medtronic...","Last week, our time-tested methodologies serve...",0.168694,Somewhat-Bullish,"[{'ticker': 'SHOP', 'relevance_score': '0.0714...",2023-07-10,11:03:00
1,"Tesla, Netflix And 2 Other Stocks Insiders Are...",The Nasdaq 100 closed lower on Friday followin...,0.093046,Neutral,"[{'ticker': 'NFLX', 'relevance_score': '0.3409...",2023-07-10,12:23:34
2,Check Out What Whales Are Doing With ORCL - Or...,Someone with a lot of money to spend has taken...,0.154769,Somewhat-Bullish,"[{'ticker': 'GS', 'relevance_score': '0.096671...",2023-07-10,16:02:28
3,Enterprise Application and Integration Market ...,Enterprise Application and Integration Market ...,0.166747,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.0292...",2023-07-11,01:45:00
4,Medical Document Management Systems Market to ...,Medical Document Management Systems Market to ...,0.154561,Somewhat-Bullish,"[{'ticker': 'KNCAF', 'relevance_score': '0.029...",2023-07-11,03:15:00


# Extracting 'ticker_sentiment' column into seperate columns

In [827]:
# Extract 'ticker' and 'relevance_score' using apply() and lambda function
df['ticker'] = df['ticker_sentiment'].apply(lambda x: x[0]['ticker'])
df['relevance_score'] = df['ticker_sentiment'].apply(lambda x: float(x[0]['relevance_score']))
df['ticker_sentiment_score'] = df['ticker_sentiment'].apply(lambda x: float(x[0]['ticker_sentiment_score']))
df['ticker_sentiment_label'] = df['ticker_sentiment'].apply(lambda x: x[0]['ticker_sentiment_label'])

df.drop('ticker_sentiment', axis=1, inplace=True)
df.head()

,title,summary,overall_sentiment_score,overall_sentiment_label,Date,Time,ticker,relevance_score,ticker_sentiment_score,ticker_sentiment_label
0,"Beat the Market Like Zacks: Shopify, Medtronic...","Last week, our time-tested methodologies serve...",0.168694,Somewhat-Bullish,2023-07-10,11:03:00,SHOP,0.071463,0.063315,Neutral
1,"Tesla, Netflix And 2 Other Stocks Insiders Are...",The Nasdaq 100 closed lower on Friday followin...,0.093046,Neutral,2023-07-10,12:23:34,NFLX,0.340915,0.032179,Neutral
2,Check Out What Whales Are Doing With ORCL - Or...,Someone with a lot of money to spend has taken...,0.154769,Somewhat-Bullish,2023-07-10,16:02:28,GS,0.096671,0.226571,Somewhat-Bullish
3,Enterprise Application and Integration Market ...,Enterprise Application and Integration Market ...,0.166747,Somewhat-Bullish,2023-07-11,01:45:00,MSFT,0.029256,0.040191,Neutral
4,Medical Document Management Systems Market to ...,Medical Document Management Systems Market to ...,0.154561,Somewhat-Bullish,2023-07-11,03:15:00,KNCAF,0.029113,0.055010,Neutral


# Dropping rows if they are not related to our ticker symbol

In [828]:
# dropping rows if they are not equal to the ticker
for index, row in df.iterrows():
    if str(df['ticker'][index]) != ticker:
        df.drop([index], axis=0, inplace=True)
        
#resetting indexes after dropping certain rows        
df.reset_index(drop=True, inplace=True)
df.head()

,title,summary,overall_sentiment_score,overall_sentiment_label,Date,Time,ticker,relevance_score,ticker_sentiment_score,ticker_sentiment_label
0,Udemy Appoints Sohaib Abbasi as Independent Ch...,Appointment adds extensive enterprise software...,0.325975,Somewhat-Bullish,2023-07-13,13:25:00,ORCL,0.068306,0.028120,Neutral
1,Udemy Appoints Sohaib Abbasi as Independent Ch...,"SAN FRANCISCO, July 13, 2023 ( GLOBE NEWSWIRE ...",0.310878,Somewhat-Bullish,2023-07-13,13:25:15,ORCL,0.063763,0.028036,Neutral
2,Largest Children's Hospital in the United Stat...,Texas Children's Hospital consolidates seven b...,0.327851,Somewhat-Bullish,2023-07-13,13:27:00,ORCL,0.856132,0.439157,Bullish
3,Oracle Unusual Options Activity For July 14 - ...,Someone with a lot of money to spend has taken...,0.155555,Somewhat-Bullish,2023-07-14,16:03:54,ORCL,0.716023,0.166114,Somewhat-Bullish
4,Oracle ( NYSE:ORCL ) Upgraded to Buy by Stoc...,StockNews.com upgraded shares of Oracle ( NYSE...,0.217602,Somewhat-Bullish,2023-07-15,09:32:47,ORCL,0.887848,0.386106,Bullish


In [829]:
df.tail()

,title,summary,overall_sentiment_score,overall_sentiment_label,Date,Time,ticker,relevance_score,ticker_sentiment_score,ticker_sentiment_label
11,Looking Into Oracle's Recent Short Interest - ...,Oracle's ORCL short percent of float has falle...,0.241618,Somewhat-Bullish,2023-07-20,19:15:27,ORCL,0.224903,0.085754,Neutral
12,StockNews.com Downgrades Oracle ( NYSE:ORCL )...,Oracle ( NYSE:ORCL - Get Free Report ) was dow...,0.266029,Somewhat-Bullish,2023-07-22,08:16:42,ORCL,0.919882,0.450401,Bullish
13,Is WisdomTree U.S. Multifactor ETF ( USMF ) ...,Smart Beta ETF report for ...,0.199316,Somewhat-Bullish,2023-07-24,10:20:06,ORCL,0.112797,0.066132,Neutral
14,Oracle Unusual Options Activity - Oracle ( NY...,Someone with a lot of money to spend has taken...,0.205614,Somewhat-Bullish,2023-07-26,20:47:38,ORCL,0.711131,0.347597,Somewhat-Bullish
15,"EGAN: Buy, Hold, or Sell: How Should Investors...",The tech sector has garnered significant inves...,0.220802,Somewhat-Bullish,2023-07-28,14:52:22,ORCL,0.202124,0.087202,Neutral


# Extracting data into a CSV

In [830]:
# import os

# current_directory = os.getcwd()
# print(current_directory)

In [831]:
# Assuming you have a DataFrame called df

# Perform operations on the DataFrame to create new columns (if needed)

# Save the DataFrame to a CSV file
# df.to_csv('ORCL_News.csv', index=False)

# Optionally, you can specify additional parameters such as delimiter and encoding
# For example:
# df.to_csv('output_file.csv', index=False, sep='\t', encoding='utf-8')


In [832]:
# # Assuming you have a DataFrame called df

# # Save the DataFrame to a specific directory
# output_path = '/path/to/your/directory/ORCL_News.csv'
# df.to_csv(output_path, index=False)

In [833]:
# # Assuming you have a DataFrame called df with updated values

# # Save the DataFrame to the CSV file in "append" mode
df.to_csv('ORCL_News.csv', mode='a', index=False, header=False)

# # Optionally, you can specify additional parameters such as delimiter and encoding
# # For example:
# # df.to_csv('AAPL_News.csv', mode='a', index=False, header=False, sep='\t', encoding='utf-8')

In [834]:
# Read the CSV file and store the data in a DataFrame
df = pd.read_csv('ORCL_News.csv')

# Display the first few rows of the DataFrame
df.tail()

,title,summary,overall_sentiment_score,overall_sentiment_label,Date,Time,ticker,relevance_score,ticker_sentiment_score,ticker_sentiment_label
566,Looking Into Oracle's Recent Short Interest - ...,Oracle's ORCL short percent of float has falle...,0.241618,Somewhat-Bullish,2023-07-20,19:15:27,ORCL,0.224903,0.085754,Neutral
567,StockNews.com Downgrades Oracle ( NYSE:ORCL )...,Oracle ( NYSE:ORCL - Get Free Report ) was dow...,0.266029,Somewhat-Bullish,2023-07-22,08:16:42,ORCL,0.919882,0.450401,Bullish
568,Is WisdomTree U.S. Multifactor ETF ( USMF ) ...,Smart Beta ETF report for ...,0.199316,Somewhat-Bullish,2023-07-24,10:20:06,ORCL,0.112797,0.066132,Neutral
569,Oracle Unusual Options Activity - Oracle ( NY...,Someone with a lot of money to spend has taken...,0.205614,Somewhat-Bullish,2023-07-26,20:47:38,ORCL,0.711131,0.347597,Somewhat-Bullish
570,"EGAN: Buy, Hold, or Sell: How Should Investors...",The tech sector has garnered significant inves...,0.220802,Somewhat-Bullish,2023-07-28,14:52:22,ORCL,0.202124,0.087202,Neutral
